In [1]:
import pandas as pd

# 최대 줄 수 설정
pd.set_option('display.max_rows', 500)
# 최대 열 수 설정
pd.set_option('display.max_columns', 500)
# 표시할 가로의 길이
pd.set_option('display.width', 1000)

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import xgboost as xgb 
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    mim  = int(x[10:12])
    sec  = int(x[12:])
    return dt.datetime(year, month, day, hour,mim,sec)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = 'C:/Users/hyeonggu/Desktop/lg 챌린지/data (2)/'

In [67]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

In [68]:
train_prob["pro_time"] = train_prob["time"].apply(make_datetime)
train_err["pro_time"] = train_err["time"].apply(make_datetime)

In [69]:
train_err.head(10)

,user_id,time,model_nm,fwver,errtype,errcode,pro_time
0,10000,20201101025616,model_3,05.15.2138,15,1,2020-11-01 02:56:16
1,10000,20201101030309,model_3,05.15.2138,12,1,2020-11-01 03:03:09
2,10000,20201101030309,model_3,05.15.2138,11,1,2020-11-01 03:03:09
3,10000,20201101050514,model_3,05.15.2138,16,1,2020-11-01 05:05:14
4,10000,20201101050515,model_3,05.15.2138,4,0,2020-11-01 05:05:15
5,10000,20201101050516,model_3,05.15.2138,26,1,2020-11-01 05:05:16
6,10000,20201101050522,model_3,05.15.2138,4,0,2020-11-01 05:05:22
7,10000,20201101060903,model_3,05.15.2138,15,1,2020-11-01 06:09:03
8,10000,20201101174754,model_3,05.15.2138,16,1,2020-11-01 17:47:54
9,10000,20201101174756,model_3,05.15.2138,4,0,2020-11-01 17:47:56


## 컴플레인 들어오기 전 데이터만 보기

In [70]:
#문제가 있는 ID만 뽑아내기
pro_id = list(set(list(train_prob["user_id"])))

In [77]:
#start = dt.datetime(2020, 11, 1, 0,0,0)
#end =  dt.datetime(2020, 11, 3, 0,0,0)
for i in tqdm(pro_id):
    if True:
        pro_time = train_prob[train_prob["user_id"] == i]["pro_time"].iloc[0]
        idx = train_err[(train_err["user_id"] == i) & (train_err["pro_time"] > pro_time)].index #이후시간
        #day = pro_time - dt.timedelta(days=2)
        #idx2 = train_err[(train_err["user_id"] == i) & (train_err["pro_time"] < day)].index #1일 전
        train_err = train_err.drop(idx)
        #train_err = train_err.drop(idx2)
    #else:
    #    idx3 = train_err[(train_err["user_id"] == i) & (train_err["pro_time"] < start)].index #1일 전
    #    idx4 = train_err[(train_err["user_id"] == i) & (train_err['pro_time'] > end)].index #1일 전

        #train_err = train_err.drop(idx3)
        #train_err = train_err.drop(idx4)        


  4%|███▏                                                                         | 206/5000 [05:43<2:13:16,  1.67s/it]


KeyboardInterrupt: 

In [75]:
train_err[train_err['user_id'] == 10004]

,user_id,time,model_nm,fwver,errtype,errcode,pro_time
3294,10004,20201101024647,model_0,04.22.1750,13,1,2020-11-01 02:46:47
3295,10004,20201101024647,model_0,04.22.1750,12,1,2020-11-01 02:46:47
3296,10004,20201101024652,model_0,04.22.1750,14,13,2020-11-01 02:46:52
3297,10004,20201101024746,model_0,04.22.1750,6,14,2020-11-01 02:47:46
3298,10004,20201101024747,model_0,04.22.1750,7,14,2020-11-01 02:47:47
3299,10004,20201101032256,model_0,04.22.1750,15,1,2020-11-01 03:22:56
3300,10004,20201101032323,model_0,04.22.1750,15,1,2020-11-01 03:23:23
3301,10004,20201101032324,model_0,04.22.1750,33,2,2020-11-01 03:23:24
3302,10004,20201101032327,model_0,04.22.1750,15,1,2020-11-01 03:23:27
3303,10004,20201101032341,model_0,04.22.1750,11,1,2020-11-01 03:23:41


In [77]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000
len(train_err)


16542113

In [78]:
pro_time = train_prob[train_prob["user_id"] == 12590]["pro_time"].iloc[0]
idx = train_err[train_err["pro_time"] > pro_time].index
idx

Int64Index([      89,       90,       91,       92,       93,       94,       95,       96,       97,       98,
            ...
            16554653, 16554654, 16554655, 16554656, 16554657, 16554658, 16554659, 16554660, 16554661, 16554662], dtype='int64', length=12183747)

In [79]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16542113/16542113 [00:46<00:00, 357210.50it/s]


(15000, 42)

In [25]:
b = dt.timedelta(days=1)

In [105]:
train_err[train_err["user_id"] == 10027]

,user_id,time,model_nm,fwver,errtype,errcode,pro_time
30706,10027,20201101075245,model_3,05.15.2138,16,1,2020-11-01 07:52:45
30707,10027,20201101075248,model_3,05.15.2138,26,1,2020-11-01 07:52:48
30708,10027,20201101075256,model_3,05.15.2138,4,0,2020-11-01 07:52:56
30709,10027,20201101084949,model_3,05.15.2138,15,1,2020-11-01 08:49:49
30710,10027,20201101084953,model_3,05.15.2138,4,0,2020-11-01 08:49:53
30711,10027,20201101165620,model_3,05.15.2138,16,1,2020-11-01 16:56:20
30712,10027,20201101165714,model_3,05.15.2138,15,1,2020-11-01 16:57:14
30713,10027,20201101165723,model_3,05.15.2138,4,0,2020-11-01 16:57:23
30714,10027,20201101170018,model_3,05.15.2138,16,1,2020-11-01 17:00:18
30715,10027,20201101180947,model_3,05.15.2138,15,1,2020-11-01 18:09:47


In [28]:
a-2*b

datetime.datetime(2021, 1, 11, 3, 1, 29, 455267)

In [52]:
train_err[train_err["errtype"] == 23].head(100)

,user_id,time,model_nm,fwver,errtype,errcode,pro_time
733,10001,20201113041408,model_2,04.33.1261,23,standby,2020-11-13 04:00:00
2996,10003,20201101224419,model_2,04.33.1185,23,connection timeout,2020-11-01 22:00:00
2997,10003,20201102002850,model_2,04.33.1185,23,connection timeout,2020-11-02 00:00:00
3005,10003,20201102103745,model_2,04.33.1185,23,connection timeout,2020-11-02 10:00:00
3012,10003,20201103035728,model_2,04.33.1185,23,connection timeout,2020-11-03 03:00:00
3016,10003,20201103101759,model_2,04.33.1185,23,connection timeout,2020-11-03 10:00:00
3038,10003,20201105214711,model_2,04.33.1185,23,connection timeout,2020-11-05 21:00:00
3040,10003,20201105214912,model_2,04.33.1185,23,connection timeout,2020-11-05 21:00:00
3091,10003,20201111014712,model_2,04.33.1261,23,active,2020-11-11 01:00:00
3191,10003,20201122173426,model_2,04.33.1261,23,active,2020-11-22 17:00:00


In [ ]:

#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정



params =      {
                 
    
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }


#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=13)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 200
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

In [ ]:
print(np.mean(auc_scores))